In [1]:
import pandas as pd
import numpy as np

In [35]:
df = pd.read_csv('Dataset/Reviews.csv')
df.head(n=1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


- we will remove all Scores equal to 3 because we assume these are neutral and did not provide us any useful information


- Add a new column called “Positivity”, where any score above 3 is encoded as a 1, indicating it was positively rated. Otherwise, it’ll be encoded as a 0, indicating it was negatively rated.

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [37]:
df = df.dropna() # Dropping all rows having Nan values
df = df[df['Score'] != 3] # Dropping all rows havinng Score=3
df['Positivity'] = np.where(df['Score'] > 3, 1, 0) # Creating a new column 'Positivity'
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Positivity
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Positivity'], random_state = 0)

print('X_train first entry: \n\n', X_train[0])
print('\n\nX_train shape: ', X_train.shape)
print('Y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('Y_test shape: ', y_test.shape)

X_train first entry: 

 I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


X_train shape:  (394329,)
Y_train shape:  (394329,)
X_test shape:  (131444,)
Y_test shape:  (131444,)


In [40]:
from sklearn.feature_extraction.text import CountVectorizer

In [41]:
count_vect = CountVectorizer()
vect = count_vect.fit(X_train)

In [44]:
print(vect.get_feature_names()[::2000])
print("Length of Vocab:",len(vect.get_feature_names()))

['00', '278', '90c', 'ahs', 'aportioned', 'b0000ijyk4', 'b000wmi6ny', 'b002tsa90m', 'bakeing', 'biodegradabl', 'bries', 'caprasite', 'chiahuahu', 'colata', 'corbination', 'dado', 'detection', 'dominate', 'elija', 'excellence', 'fim', 'frieda', 'gleeful', 'gumshoed', 'himsef', 'immunosuppresed', 'ironbeer', 'keyed', 'length', 'macciaroni', 'medjugorje', 'momentum', 'nemo', 'oesk', 'pacaka', 'perritos', 'popluar', 'prs', 'reapplied', 'resourceful', 'saffron', 'sen', 'skimming', 'speculatius', 'stripes', 'tabascos', 'thoughtful', 'trot', 'unsafely', 'volcanic', 'wirehair', 'zits']
Length of Vocab: 102135


In [48]:
X_train_vectorized = vect.transform(X_train)

In [49]:
from sklearn.linear_model import LogisticRegression

In [50]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

c:\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
from sklearn.metrics import roc_auc_score
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.8510832937056847


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df = 5).fit(X_train)
print(len(vect.get_feature_names()))

35289


In [53]:
X_train_vectorized = vect.transform(X_train)

In [63]:
print(X_train_vectorized.shape)
print(type(X_train_vectorized))

(394329, 35289)
<class 'scipy.sparse.csr.csr_matrix'>


In [65]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

c:\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.8511620421199818


In [66]:
print(model.predict(vect.transform(['The candy is not good, I will never buy them again','The candy is not bad, I will buy them again'])))

[1 0]


In [68]:
vect = TfidfVectorizer(min_df = 5,ngram_range = (1,2)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
len(vect.get_feature_names())

528082

In [69]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

c:\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.8888393636731423


In [70]:
print(model.predict(vect.transform(['The candy is not good, I would never buy them again','The candy is not bad, I will buy them again'])))

[0 1]


In [72]:
print(model.predict(vect.transform(['Good taste! Highly recommended'])))

[1]


#### Saving Model

In [73]:
import pickle

In [74]:
filename = 'tfidf_logistic_regression_amzn_food_review_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [75]:
# Saving Vocab
pickle.dump(vect.vocabulary_,open("feature.pkl","wb"))

In [76]:
print(type(vect))

<class 'sklearn.feature_extraction.text.TfidfVectorizer'>


In [77]:
import joblib

In [78]:
joblib.dump(vect, "vectorizer.pkl")

['vectorizer.pkl']

In [79]:
vectorizer = joblib.load("vectorizer.pkl")